In [ ]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train = pd.read_csv(train_file_path,delimiter= '\t', names=['ham/spam','sentence'])
train_data = train['sentence']
train_labels = train['ham/spam']

test = pd.read_csv(test_file_path,delimiter= '\t', names=['ham/spam','sentence'])
test_data = test['sentence']
test_labels = test['ham/spam']


n_words_train = train_data.apply(lambda x: len(x.split()))
n_words_test = test_data.apply(lambda x: len(x.split()))
total_sum = (n_words_train.sum() + n_words_test.sum())
average = total_sum / (len(train_data) + len(test_data))
average= round(average,0)
print(total_sum)
print(average)

In [ ]:
# tokenizing of the input SMS messages

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)

train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

padded_train_sequences = pad_sequences(train_sequences, maxlen=int(average), padding='post', truncating='post')
padded_test_sequences = pad_sequences(test_sequences, maxlen=int(average), padding='post', truncating='post')

train_data_pad = pd.DataFrame(padded_train_sequences)
test_data_pad = pd.DataFrame(padded_test_sequences)

unique_tokens = set()

for sentence in train_sequences:
    unique_tokens.update(sentence)

for sentence in test_sequences:
    unique_tokens.update(sentence)

input_dim = len(unique_tokens)
print(input_dim)

In [ ]:
# encoding of the labels into numerical values

mapping = {'ham': 0, 'spam':1}

train_labels_map = train_labels.map(mapping)
test_labels_map = test_labels.map(mapping)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

model = Sequential([
                    Embedding(input_dim= input_dim + 1, output_dim=32, mask_zero=True),
                    LSTM(32),
                    Dense(10,activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
                  ])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(padded_train_sequences, train_labels_map,
          epochs=10,
          batch_size=32)

In [ ]:
test_loss, test_acc = model.evaluate(padded_test_sequences, test_labels_map)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

    pred_text_tokenized = tokenizer.texts_to_sequences([pred_text])

    padded_pred_text = pad_sequences(pred_text_tokenized, maxlen=int(average), padding='post', truncating='post')

    prediction = model.predict(padded_pred_text)

    if prediction >=0.5:
      prediction = [prediction, 'spam']
    else:
      prediction = [prediction, 'ham']

    return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
